In [2]:
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf

from scipy.stats import norm, chi2

from statsmodels.tsa.arima_process import arma_generate_sample # Import the necessary function

In [3]:
# 1. Generate a synthetic time series using arma_generate_sample
np.random.seed(45) # Same seed for consistent example
n_samples = 200

# AR(1) coefficient
ar_coefficient = 0.25

# For yt = ar_coefficient * y(t-1) + epsilon_t, ar_params should be [1, -ar_coefficient]
ar_params = np.array([1, -ar_coefficient])
ma_params = np.array([1]) # No MA component for a pure AR(1) process

# Generate the time series
time_series = arma_generate_sample(ar_params, ma_params, n_samples)

In [4]:
# 2. Calculate the Autocorrelation Function (ACF) values and confidence intervals
nlags = 20 # Number of lags to consider
acf_values = acf(time_series, nlags=nlags, fft=False)

# Approximate 95% confidence interval for ACF (for white noise)
alpha_ci = 0.05
conf_int_bound = norm.ppf(1 - alpha_ci / 2) / np.sqrt(len(time_series))

In [5]:
# 3. Perform the Ljung-Box test
ljung_box_results = acorr_ljungbox(time_series, lags=np.arange(1, nlags + 1), return_df=True)
p_values_lb = ljung_box_results['lb_pvalue']
lb_stats = ljung_box_results['lb_stat'] # Ljung-Box test statistic
lags_lb = ljung_box_results.index


# Calculate chi-squared critical values for each lag
alpha_lb = 0.05 # Significance level for Ljung-Box test
critical_values_chi2 = [chi2.ppf(1 - alpha_lb, df=lag) for lag in lags_lb]

In [6]:
# 4. Create three subplots
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("Simulated Time Series (AR(1) Component)",
                                    "Autocorrelation Function (ACF) vs. Ljung-Box P-values",
                                    "Ljung-Box Test Statistic vs. Chi-Squared Critical Value"),
                    vertical_spacing=0.1,
                    specs=[[{"type": "xy"}],
                           [{"type": "xy", "secondary_y": True}], # For ACF and P-values
                           [{"type": "xy"}]]) # For Test Statistic


# --- Top subplot: Time Series ---
fig.add_trace(go.Scatter(y=time_series, mode='lines', name='Simulated Series',
                         line=dict(color='lightsteelblue'), showlegend=True),
              row=1, col=1)
fig.update_xaxes(title_text="Time Point", row=1, col=1)
fig.update_yaxes(title_text="Value", row=1, col=1)

# --- Middle subplot: ACF Plot (Primary Y-axis) and Ljung-Box P-values (Secondary Y-axis) ---
fig.add_trace(go.Bar(x=np.arange(1, nlags + 1), y=acf_values[1:], name='ACF (Magnitude)',
                         marker_color='royalblue', showlegend=True),
              row=2, col=1, secondary_y=False)

# Confidence intervals for ACF
fig.add_shape(type="line",
              x0=0.5, y0=conf_int_bound, x1=nlags + 0.5, y1=conf_int_bound,
              line=dict(color="crimson", width=1.5, dash="dash"),
              row=2, col=1, secondary_y=False)
fig.add_shape(type="line",
              x0=0.5, y0=-conf_int_bound, x1=nlags + 0.5, y1=-conf_int_bound,
              line=dict(color="crimson", width=1.5, dash="dash"),
              row=2, col=1, secondary_y=False)
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', # Dummy trace for legend entry
                         line=dict(color="crimson", width=1.5, dash="dash"),
                         name='ACF 95% Confidence Interval', showlegend=True),
              row=2, col=1, secondary_y=False)


# Ljung-Box P-values (Secondary Y-axis for middle subplot)
fig.add_trace(go.Scatter(x=lags_lb, y=p_values_lb, mode='markers+lines', name='Ljung-Box P-value',
                         marker=dict(color='darkgreen', size=7, symbol='circle'),
                         line=dict(color='darkgreen', dash='dot', width=1),
                         hoverinfo='x+y',
                         hovertemplate='<b>Lag:</b> %{x}<br><b>Ljung-Box P-value:</b> %{y:.3f}<extra></extra>',
                         showlegend=True),
              row=2, col=1, secondary_y=True)

# Significance level for Ljung-Box P-values (0.05)
fig.add_shape(type="line",
              x0=0.5, y0=0.05, x1=nlags + 0.5, y1=0.05,
              line=dict(color="darkorange", width=1.5, dash="longdash"),
              row=2, col=1, secondary_y=True)
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', # Dummy trace for legend entry
                         line=dict(color="darkorange", width=1.5, dash="longdash"),
                         name='Ljung-Box 0.05 Significance Level', showlegend=True),
              row=2, col=1, secondary_y=True)


fig.update_xaxes(title_text="Lag", row=2, col=1)
fig.update_yaxes(title_text="ACF Value", row=2, col=1, secondary_y=False, range=[-0.3, 0.3])
fig.update_yaxes(title_text="Ljung-Box P-value", row=2, col=1, secondary_y=True, range=[0, 1.0])

# --- Bottom subplot: Ljung-Box Test Statistic ---
fig.add_trace(go.Scatter(x=lags_lb, y=lb_stats, mode='markers+lines', name='Ljung-Box Test Statistic',
                         marker=dict(color='purple', size=7, symbol='square'),
                         line=dict(color='purple', dash='solid', width=1),
                         hoverinfo='x+y',
                         hovertemplate='<b>Lag:</b> %{x}<br><b>Test Stat:</b> %{y:.2f}<extra></extra>',
                         showlegend=True),
              row=3, col=1)

fig.add_trace(go.Scatter(x=lags_lb, y=critical_values_chi2, mode='lines', name='Chi-Squared Critical Value (df=Lag)',
                         line=dict(color='gray', width=1.5, dash='dashdot'),
                         hoverinfo='x+y',
                         hovertemplate='<b>Lag:</b> %{x}<br><b>Critical Value:</b> %{y:.2f}<extra></extra>',
                         showlegend=True),
              row=3, col=1)

fig.update_xaxes(title_text="Lag", row=3, col=1)
fig.update_yaxes(title_text="Test Statistic Value", row=3, col=1)


# Update overall layout
fig.update_layout(title_text="Comprehensive Comparison: Ljung-Box Test vs. Visual ACF Inspection",
                  showlegend=True,
                  height=950, # Increased height for third subplot
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5, font=dict(size=10)))

# Add a concise explanatory annotation
fig.add_annotation(
    text="**Interpretation Summary:**<br>"
         "Lag 1 shows strong significance across all metrics (ACF, P-value, Test Stat).<br>"
         "For other lags, methods largely agree on insignificance.<br>"
         "Ljung-Box is key for subtle, cumulative dependencies not always visible in ACF.",
    xref="paper", yref="paper",
    x=0.0, y=-0.15, # Position at bottom-left relative to overall figure
    showarrow=False,
    font=dict(size=10, color="darkslategray"),
    align="left",
    bgcolor="white",
    bordercolor="lightgray",
    borderwidth=1,
    borderpad=5
)

fig.show()